In [ ]:
from pathways.scenarios import run_scenarios, load_scenario_table
from pathways.simulation import load_configuration
from pathways.outputs import save_scenario_result_to_pandas

In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

%matplotlib inline

In [ ]:
from pathlib import Path
datadir = Path("tests/test_summaries")
# Set a directory for the use case results here
resultsdir = Path("use_cases")
# Make sure the directory exists
resultsdir.mkdir(exist_ok=True)

## Load scenario tables for use cases
Note that these use cases recreate consignments from historical AQIM inspection data (aqim_box_insp_unit.csv). To run this notebook, the inspection data csv must be saved locally or a different method for generating the consignments should be used. You can edit the scenario tables to use the parameter consignment generator or use a csv of other AQIM or F280 inspection records.

In [ ]:
basic_config = load_configuration(datadir / "config.yml")
rate_scenario_table = load_scenario_table(datadir / "contamination_rate_estimation.csv")
inspection_scenario_table = load_scenario_table(datadir / "inspection_scenarios.csv")
consignment_scenario_table = load_scenario_table(datadir / "consignment_scenarios.csv")

## Use Case 1: Estimate contamination rates from high quality inspection data

When inspection data obtained with known statistically valid inspection methods are available, the simulation can be used to estimate the consignment contamination rates by recreating the inspections and calibrating the contamination configuration until similar inspection outcomes are achieved.

In the example below, data from AQIM inspections of cut flower consignments are used to estimate possible contamination rate probability distributions. Note that the contamination rate distribution parameters were estimated by running the simulation, checking the failure rate, and adjusting the distribution parameters until the failure rate matched the AQIM data.

In [ ]:
num_consignments = 3313
fitted_contamination_rate_results = run_scenarios(
    config=basic_config,
    scenario_table=rate_scenario_table,
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
df_fitted = save_scenario_result_to_pandas(
    fitted_contamination_rate_results,
    config_columns=[
        "name",
        "consignment name",
        "inspection name",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
    ],
    result_columns=[
        "true_contamination_rate",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
        "avg_boxes_opened_completion",
    ],
)

In [ ]:
df_fitted['failure rate'] = df_fitted["intercepted"] / num_consignments

In [ ]:
# Format dataframe
column_names = ["consignment name", "inspection name", "beta parameters", "contaminant arrangement", "cluster distribution", "infested boxes per cluster", "simulated contamination rate (mean)", "failure rate"]
df_contamination_pretty = df_fitted.iloc[:,[1,2,3,4,5,6,7,13]].copy()

df_contamination_pretty.columns = column_names
df_contamination_pretty.iloc[:,6] = df_contamination_pretty.iloc[:,6].round(decimals=4)
df_contamination_pretty.iloc[:,7] = df_contamination_pretty.iloc[:,7].round(decimals=4)
df_contamination_pretty

In [ ]:
# Save results to csv
df_contamination_pretty.to_csv(resultsdir / "contamination_rate_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this chunk.
#df_contamination_pretty = pd.read_csv(resultsdir / "contamination_rate_results.csv")

## Use Case 2: Measure the effect of deviations from sampling protocols

We used the calibrated contamination rate distribution with mean 0.0027 and standard deviation 0.0282 with a clustered contaminant arrangement to run sampling scenarios with fixed consignment assumptions. The outcomes of these scenarios provide information about the relative impacts of changes to inspection protocols. 

In [ ]:
# Hiding very long output for this cell (many printed messages related to clusters)
#%%capture capt
num_consignments = 3313
inspection_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=inspection_scenario_table,
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)
# uncomment to print output if desired
#capt.show()

In [ ]:
df_inspections = save_scenario_result_to_pandas(
    inspection_scenario_results,
    config_columns=[
        "name",
        "inspection/unit",
        "inspection/sample_strategy",
        "inspection/proportion/value",
        "inspection/hypergeometric/detection_level",
        "inspection/selection_strategy",
        "inspection/cluster/cluster_selection",

    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "avg_items_inspected_completion",
        "avg_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
    ],
)

In [ ]:
df_inspections['failure rate'] = df_inspections["intercepted"] / num_consignments
contaminated_consignments = df_inspections["false_neg"] + df_inspections["intercepted"]
df_inspections["interception rate"] = df_inspections["intercepted"] / contaminated_consignments
df_inspections["% missed contaminants"] = (df_inspections["total_missed_contaminants"] / (df_inspections["total_missed_contaminants"] + df_inspections["total_intercepted_contaminants"])) * 100

In [ ]:
# Format dataframe
column_names = ["name", "inspection unit", "sample strategy", "sample parameter", "selection strategy", "cluster selection", "avg contamination rate", "max missed contamination rate", "avg missed contamination rate", "max intercepted contamination rate", "avg intercepted contamination rate", "boxes opened completion", "boxes opened detection", "items inspected completion", "items inspected detection", "missed", "intercepted", "missed contaminants", "intercepted contamininants", "failure rate", "interception rate", "% missed contaminants", "sample size method", "selection method"]

In [ ]:

df_inspections_pretty = df_inspections.loc[:, df_inspections.columns != 'inspection/hypergeometric/detection_level'].copy()
hypergeometric_parameters = df_inspections.iloc[[0,1,2,3,4,5,6,7,12,13,14,15],4]
df_inspections_pretty.iloc[[0,1,2,3,4,5,6,7,12,13,14,15],3] = hypergeometric_parameters
df_inspections_pretty.iloc[:,3] = df_inspections_pretty.iloc[:,3].astype(str)

df_inspections_pretty["sample size method"] = df_inspections_pretty[['inspection/sample_strategy', 'inspection/proportion/value']].agg(' '.join, axis=1)
df_inspections_pretty["selection method"] = df_inspections_pretty[["inspection/unit", 'inspection/selection_strategy', 'inspection/cluster/cluster_selection']].agg(' '.join, axis=1)

df_inspections_pretty.columns = column_names
df_inspections_pretty.iloc[:,6:11] = df_inspections_pretty.iloc[:,6:11].round(decimals=4)
df_inspections_pretty.iloc[:,11:19] = df_inspections_pretty.iloc[:,11:19].astype(int)
df_inspections_pretty.iloc[:,19:22] = df_inspections_pretty.iloc[:,19:22].round(decimals=4)
df_inspections_pretty.iloc[:18,[0,1,21,22,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
df_inspections_pretty

In [ ]:
# Save results to csv
df_inspections_pretty.to_csv(resultsdir / "inspection_scenario_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this chunk.
#df_inspections_pretty = pd.read_csv(resultsdir / "inspection_scenario_results.csv")

In [ ]:
colors = {"hypergeometric 0.1": "#1f78b4", "hypergeometric 0.05": "#a6cee3", "proportion 0.02":"#b2df8a"}
patch_1 = mpatches.Patch(color="#a6cee3", label="hypergeometric 0.05")
patch_2 = mpatches.Patch(color="#1f78b4", label="hypergeometric 0.1")
patch_3 = mpatches.Patch(color="#b2df8a", label="proportion 0.02")

In [ ]:
plt.figure(figsize=(18, 18), dpi=150)
plt.subplot(221)
plt.subplots_adjust(wspace=0.65, left=0.1,right=0.9, top=0.93, bottom=0.05, hspace=0.16)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["interception rate"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Interception Rate", fontsize=24)
plt.ylabel("inspection method", fontsize=20)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(222)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["avg missed contamination rate"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rates", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(ticks=[0,0.01,0.02,0.03], fontsize=18)

plt.subplot(223)
plt.subplots_adjust(wspace=0.65,left=0.22,right=0.95)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["boxes opened completion"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Boxes Opened per Consignment", fontsize=24)
plt.ylabel("inspection method", fontsize=20)
plt.xlabel("boxes", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(224)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["items inspected completion"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Items Inspected per Consignment", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.legend(handles=[patch_1,patch_2,patch_3], loc = "lower right", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(fontsize=18)
plt.suptitle("Inspection Scenarios", fontsize=28)
plt.savefig(resultsdir / "inspection_scenario_plots.png")
plt.show()


##  Use Case 3: Measure the effect of changes in consignment characteristics

We used the simulation with fixed inspection assumptions to answer questions about how inspection outcomes change with changes in consignments. Using the AQIM inspection protocol (box unit, hypergeometric sample with 0.1 detection level and 0.95 confidence level, random selection), we simulated multiple scenarios to reflect the following consignment scenarios:

* 10,000,000 items packaged using three cargo scenarios: Maritime scenario with large consignments (100 - 160 boxes) with 700 items per box, Air scenario with mid-sized consignments (20 - 100 boxes) with 200 items per box, and direct-to-consumer scenario with very small consignments (1 - 50 boxes) with 100 items per box.
* Changes in contamination rate variability
* Changes in contaminant arrangement (random vs clustered)

### First, run the packaging scenarios. Each requires a different number of consignments per simulation to contain 10,000,000 items.

In [ ]:
num_consignments = 833
air_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[0:1],
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
num_consignments = 110
maritime_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[1:2],
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
num_consignments = 4000
dtc_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[2:3],
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
df_consignments_10M = save_scenario_result_to_pandas(
    air_scenario_results+maritime_scenario_results+dtc_scenario_results,
    config_columns=[
        "name",
        "consignment name",
        "consignment/boxes/min",
        "consignment/boxes/max",
        "consignment/items_per_box/default",
        "contamination/contamination_unit",
        "contamination/contamination_rate/distribution",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
        "contamination/clustered/random/cluster_item_width",
    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "pct_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "pct_boxes_opened_detection",
        "avg_items_inspected_completion",
        "pct_items_inspected_completion",
        "avg_items_inspected_detection",
        "pct_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
        "num_boxes",
        "num_items",
    ],
)

In [ ]:
df_consignments_10M['failure rate'] = df_consignments_10M["intercepted"] / num_consignments
contaminated_consignments = df_consignments_10M["false_neg"] + df_consignments_10M["intercepted"]
df_consignments_10M["interception rate"] = df_consignments_10M["intercepted"] / contaminated_consignments
df_consignments_10M["contaminated_consignments"] = contaminated_consignments
df_consignments_10M["% missed contaminants"] = (df_consignments_10M["total_missed_contaminants"] / (df_consignments_10M["total_missed_contaminants"] + df_consignments_10M["total_intercepted_contaminants"])) * 100

In [ ]:
# Format dataframe 
column_names = ["name", "consignment name", "items per box", "contamination unit", "contamination parameters", "contaminant arrangement", "cluster distribution", "contaminated units per cluster", "cluster width", "avg contamination rate", "avg missed contamination rate", "avg intercepted contamination rate", "avg boxes opened per inspection", "pct boxes opened per simulation", "avg items inspected per inspection", "pct items inspected per simulation", "missed contaminants", "intercepted contamininants", "total boxes", "total items", "interception rate", "contaminated_consignments", "% missed contaminants"]
df_consignments_pretty_10M = df_consignments_10M

In [ ]:
df_consignments_pretty_10M.iloc[:,12:35] = df_consignments_pretty_10M.iloc[:,12:35].round(decimals=3)
df_consignments_pretty_10M = df_consignments_pretty_10M.iloc[:,[0,1,4,5,7,8,9,10,11,12,14,16,17,18,21,22,27,28,29,30,32,33,34]]
df_consignments_pretty_10M.columns = column_names
df_consignments_pretty_10M

In [ ]:
# Save results to csv
df_consignments_pretty_10M.to_csv(resultsdir / "cargoconfig_scenario_10M_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this cell.
#df_consignments_pretty_10M = pd.read_csv(resultsdir / "cargoconfig_scenario_10M_results.csv")

In [ ]:
plt.figure(figsize=(16, 5), dpi=300)
plt.subplot(221)
plt.subplots_adjust(bottom=0.14,top=0.81, left=0.2, right=0.97, wspace=0.65, hspace=1)
plt.barh(df_consignments_pretty_10M["name"], df_consignments_pretty_10M["interception rate"], color="#b2df8a")
plt.title("Interception Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.ylabel("cargo type", fontsize=18, labelpad=10)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_10M["consignment name"], fontsize=20)
plt.xticks(ticks=[0.0,0.2,0.4,0.6,0.8],fontsize=18)
plt.subplot(222)
plt.barh(df_consignments_pretty_10M["name"], df_consignments_pretty_10M["avg missed contamination rate"], color="#b2df8a")
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_10M["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.001,0.002,0.003,0.004], fontsize=18)

plt.subplot(223)
plt.barh(df_consignments_pretty_10M["name"], df_consignments_pretty_10M["avg items inspected per inspection"], color="#b2df8a")
plt.title("Items Inspected per Consignment", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.ylabel("cargo type", fontsize=18, labelpad=10)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_10M["consignment name"], fontsize=20)
plt.xticks(ticks=[0,4000,8000,12000,16000], fontsize=18)
plt.subplot(224)
plt.barh(df_consignments_pretty_10M["name"], df_consignments_pretty_10M["pct items inspected per simulation"], color="#b2df8a")
plt.title("% Items Inspected per Scenario", fontsize=24)
plt.xlabel("% items", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_10M["consignment name"], fontsize=20)
plt.xticks(fontsize=18)

plt.suptitle("Cargo Packaging Scenarios", fontsize=28)
plt.savefig(resultsdir / "cargo_config_scenario_10M_plots.png")
plt.show()

### Run contaminant arrangement and rate variability scenarios

In [ ]:
num_consignments = 3313
consignment_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[3:9],
    seed=42,
    num_simulations=100,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
df_consignments = save_scenario_result_to_pandas(
    consignment_scenario_results,
    config_columns=[
        "name",
        "consignment name",
        "consignment/boxes/min",
        "consignment/boxes/max",
        "consignment/items_per_box/default",
        "contamination/contamination_unit",
        "contamination/contamination_rate/distribution",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
        "contamination/clustered/random/cluster_item_width",
    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "pct_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "pct_boxes_opened_detection",
        "avg_items_inspected_completion",
        "pct_items_inspected_completion",
        "avg_items_inspected_detection",
        "pct_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
    ],
)

In [ ]:
df_consignments['failure rate'] = df_consignments["intercepted"] / num_consignments
contaminated_consignments = df_consignments["false_neg"] + df_consignments["intercepted"]
df_consignments["interception rate"] = df_consignments["intercepted"] / contaminated_consignments
df_consignments["contaminated_consignments"] = contaminated_consignments
df_consignments["% missed contaminants"] = (df_consignments["total_missed_contaminants"] / (df_consignments["total_missed_contaminants"] + df_consignments["total_intercepted_contaminants"])) * 100

In [ ]:
# Format dataframe 
column_names = ["name", "consignment name", "items per box", "contamination unit", "contamination parameters", "contaminant arrangement", "cluster distribution", "contaminated units per cluster", "cluster width", "avg contamination rate", "avg missed contamination rate", "avg intercepted contamination rate", "avg boxes opened per inspection", "pct box opened per simulation", "avg items inspected per inspection", "pct items inspected per simulation", "missed contaminants", "intercepted contamininants", "interception rate", "contaminated_consignments", "% missed contaminants"]
df_consignments_pretty = df_consignments

In [ ]:
df_consignments_pretty.iloc[:,12:17] = df_consignments_pretty.iloc[:,12:17].round(decimals=4)
df_consignments_pretty.iloc[:,17:29] = df_consignments_pretty.iloc[:,17:29].astype(int)
df_consignments_pretty.iloc[:,29:31] = df_consignments_pretty.iloc[:,29:31].round(decimals=4)
df_consignments_pretty.iloc[:,[31]] = df_consignments_pretty.iloc[:,[31]].astype(int)
df_consignments_pretty.iloc[:,[32]] = df_consignments_pretty.iloc[:,[32]].round(decimals=4)
df_consignments_pretty = df_consignments_pretty.iloc[:,[0,1,4,5,7,8,9,10,11,12,14,16,17,18,21,22,27,28,30,31,32]]
df_consignments_pretty.columns = column_names
df_consignments_pretty

In [ ]:
# Save results to csv
df_consignments_pretty.to_csv(resultsdir / "consignment_scenario_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this chunk.
#df_consignments_pretty = pd.read_csv(resultsdir / "consignment_scenario_results.csv")

In [ ]:
df_contamination_rate_scenarios = df_consignments_pretty.loc[0:5,:]
df_contaminant_arrangement_scenarios = df_consignments_pretty.loc[9:,:]

In [ ]:
colors = {"item": "#b2df8a", "box": "#1f78b4"}
patch_1 = mpatches.Patch(color="#b2df8a", label="item")
patch_2 = mpatches.Patch(color="#1f78b4", label="box")

plt.figure(figsize=(16, 4), dpi=300)
plt.subplot(121)
plt.subplots_adjust(bottom=0.25,top=0.77, left=0.08, right=0.97)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["interception rate"], color=df_contamination_rate_scenarios['contamination unit'].replace(colors))
plt.title("Interception Rate", fontsize=24)
plt.xlabel("rate", fontsize=18)
plt.ylabel("rate variability", fontsize=18, labelpad=10)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(122)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["avg missed contamination rate"], color=df_contamination_rate_scenarios['contamination unit'].replace(colors))
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=18)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.002,0.004,0.006], fontsize=18)
plt.suptitle("Contamination Rate Variability Scenarios", fontsize=28)
plt.legend(handles=[patch_2,patch_1], loc = "lower right", fontsize=20, borderpad=0.2, labelspacing=0.2)

plt.savefig(resultsdir / "rate_variability_scenario_plots.png")
plt.show()

In [ ]:
colors = {"item": "#b2df8a", "box": "#1f78b4"}
patch_1 = mpatches.Patch(color="#b2df8a", label="item")
patch_2 = mpatches.Patch(color="#1f78b4", label="box")

plt.figure(figsize=(16, 3.2), dpi=300)
plt.subplot(121)
plt.subplots_adjust(bottom=0.25,top=0.72, left=0.18, right=0.97, wspace=0.4)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["interception rate"], color=df_contaminant_arrangement_scenarios['contamination unit'].replace(colors))
plt.title("Interception Rate", fontsize=24)
plt.xlabel("rate", fontsize=18)
plt.ylabel("contaminant \narrangement", fontsize=18, labelpad=10)
plt.yticks(ticks=np.arange(4),labels=df_contaminant_arrangement_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(122)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["avg missed contamination rate"], color=df_contaminant_arrangement_scenarios['contamination unit'].replace(colors))
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=18)
plt.yticks(ticks=np.arange(4),labels=df_contaminant_arrangement_scenarios["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.002,0.004,0.006], fontsize=18)
plt.suptitle("Contaminant Arrangement Scenarios", fontsize=28)
plt.legend(handles=[patch_2,patch_1], loc="right", fontsize=20, borderpad=0.2, labelspacing=0.2)

plt.savefig(resultsdir / "contaminant_arrangement_scenarios_plots.png")
plt.show()